### Performing parameter studies with PINNs in TorchPhysics
We now consider the wave equation

\begin{align*}
    \partial_t^2 u &= c \, \partial_x^2 u, &&\text{ in } I_x \times I_t, \\
    u &= 0 , &&\text{ on } \partial I_x \times I_t, \\
    (\partial_t u)(\cdot, 0) &= 0 , &&\text{ in } I_x, \\
    u(\cdot, 0) &= \sin(x) , &&\text{ in } I_x,
\end{align*}

with $I_x = [0, 2\pi]$ and $I_t = [0, 5]$. We aim to solve the equation for all $c \in [0.2, 2.5]$.

In this notebook, we provided less guidance to repeat the contents of yesterday.

In [ ]:
# This block is for GPU selection. Please execute.
import pathlib
import os
user = int(str(pathlib.Path().resolve())[22:24])
os.environ["CUDA_VISIBLE_DEVICES"]= str(user % 4)

In [ ]:
import torchphysics as tp
import pytorch_lightning as pl
import torch
import math

# Here all parameters are defined:
t_min, t_max = 0.0, 5.0
x_min, x_max = 0.0, 2 * math.pi
c_min, c_max = 0.2, 2.5

# Number of training points 
N_pde = 25000
N_boundary = 5000
N_initial = 5000

# Training parameters
train_iterations = 10000
learning_rate = 8.e-4

In [2]:
### TODO: Implement the spaces
X = ...
T = ...
U = ...
C = ...

### TODO: Define the domain, time interval and parameter range as torchphysics domains:
I_x = ...
I_t = ...
I_c = ...

### TODO: Create sampler for the PDE condition. Hint: sample from a product domain including x, t and c
pde_sampler = ...

### TODO: Create a random uniform sampler for the Dirichlet boundary condition:
boundary_sampler = ...

### TODO: Create a sampler for the two initial conditions:
initial_sampler = ...

In [3]:
### TODO: Create the neural network for the solution u, depending on the parameter c.
###       The model of u should contain 3 hidden layers with 50 neurons each and should have
###       X*T*C as an input space.
model_u = ...

In [4]:
### TODO: Define condition for the PDE:
def pde_residual(c, u, t, x):
    return ...

pde_condition = ...

In [5]:
### TODO: Define the Dirichlet boundary condition:
def dirichlet_residual(u):
    return u
dirichlet_condition = ...

In [6]:
### TODO: Define both initial conditions: one that sets the values and one that sets the derivatives, each at t=0.

def initial_residual(u, x):
    return ...
initial_condition = ...

def initial_derivative_residual(u, t):
    return ...
initial_derivative_condition = ...

In [ ]:
### The optimizer is already implemented. Just fill in the conditions that should be optimized and you can start the training.

optim = tp.OptimizerSetting(optimizer_class=torch.optim.Adam, lr=learning_rate)
solver = tp.solver.Solver(train_conditions=[..., ..., ..., ...], optimizer_setting=optim)


trainer = pl.Trainer(devices=1, accelerator="gpu",
                     max_steps=train_iterations,
                     logger=False,
                     benchmark=True,
                     enable_checkpointing=False)

trainer.fit(solver)

In [ ]:
### We can also plot the solution that we learned
plot_domain = tp.domains.Parallelogram(X*T, [x_min, t_min], [x_max, t_min], [x_min, t_max])
plot_sampler = tp.samplers.PlotSampler(plot_domain, 1000, data_for_other_variables={'c': 0.2})
fig = tp.utils.plot(model_u, lambda u: u, plot_sampler, plot_type="contour_surface")

In [ ]:
# Or an animation:
anim_sampler = tp.samplers.AnimationSampler(I_x, I_t, 200, n_points=250, data_for_other_variables={'c': 0.5})
fig, anim = tp.utils.animate(model_u, lambda u: u, anim_sampler, ani_speed=40)
anim.save('wave-eq.gif')
# You should be able to see a tab with a folder on the left side of your screen. There you should find the gif and can watch it.